#### Componentes Fundamentais:

- **Models (Modelos):** LangChain serve como uma interface padrão que permite interações com uma ampla gama de Grandes Modelos de Linguagem (LLMs).

- **Chains (Cadeias):** Como seu nome implica, _cadeias_ são o núcleo dos fluxos de trabalho do LangChain. Combinam LLMs com outros componentes, criando aplicativos por meio da execução de uma sequência de funções.

- **Prompts (Instruções):** Os prompts são as instruções apresentadas a um LLM. Geralmente, a "arte" de redigir prompts que efetivamente entregam o contexto necessário para que o LLM interprete a entrada e a saída da estrutura da maneira mais útil para você é chamada de engenharia de prompt.

- **Indexes (Índices):** Para realizar determinadas tarefas, as LLMs precisarão acessar fontes de dados externas específicas não incluídas em seu conjunto de dados de treinamento, como documentos internos, e-mails ou conjuntos de dados. LangChain refere-se coletivamente a essa documentação externa como “índices"."

- **Memory (Memória):** Por padrão, os LLMs não têm memória de longo prazo de conversas anteriores (a menos que o histórico do chat seja usado como entrada para uma consulta). O LangChain soluciona esse problema com utilitários simples para adicionar memória a um sistema, com opções que vão desde a retenção total de todas as conversas até a retenção de um resumo da conversa até a retenção das _n_ trocas mais recentes.

- **Agents/Tools (Agentes/Ferramentas):** Os agentes do LangChain podem usar um determinado modelo LLM como um "mecanismo de raciocínio" para determinar quais ações tomar. Ao criar uma cadeia para um agente, as entradas contêm:

	- uma lista de ferramentas disponíveis para serem aproveitadas.
	- entrada do usuário (como prompts e consultas).
	- quaisquer etapas relevantes executadas anteriormente.

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)

model = "gpt-4o-mini"

In [ ]:

response = client.responses.create(
    model=model,
    input="Fale sobre SOLID",
    store=True,
)

print(response.output_text);

In [ ]:
 
response = client.completions.create(
  model=model,
  prompt="Crie uma canção contendo apenas uma estrofe."
)

message = (response.choices[0].text)
print(message)

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Conte uma piada sobre programação"},
    ]
)

print(response)
message = (response.choices[0].message.content)
print(message)

### Roles

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": "Você é um assistente de investimentos fictício."},
    {"role": "user", "content": "Qual é o melhor investimento de baixo risco que você recomenda para este ano?"}
  ]
)

message = (response.choices[0].message.content)
print(message)

## Hiper Parâmetros

In [ ]:
# frequency_penalty - justa a probabilidade de repetição de frase/palavras, reduzindo a redundância(positivo) ou aumentando a repetição(negativo)-2 e 2, padrão é 0
# presence_penalty - penaliza a inclusão de palavras novas, incentivando a diversidade
# max_tokens - define o número máximo de tokens na resposta, limitando o tamanho da resposta
# temperature - controla a aleatoriedade das respostas, mais baixo (0.0) gera respostas mais previsíveis, enquanto valores mais altos (até 1.0) geram respostas mais criativas e variadas
# n - número de respostas a serem geradas, padrão é 1
# seed -  valor de inicialização da API, aumenta a probabilidade de você obter a mesma resposta para a mesma pergunta, útil para testes e depuração
# stop - ele vai fazer com que o modelo pare de gerar texto quando encontrar uma sequência específica, útil para controlar o final da resposta

response = client.chat.completions.create(
  model=model,
  frequency_penalty=1,
  presence_penalty = 1,
  temperature =  1 ,
  max_tokens=500,
  n = 2 ,
  seed = 123,
  #stop = ["shadows","mortal "],
  messages=[
    {"role": "system", "content": "Você é um poeta deprimido e desiludido."},
    {"role": "user", "content": "Componha um poema no máximo 5 linhas e 3 parágrafos."}
  ]
)


print(response.choices[0].message.content)
print(" -------------------------")
print(response.choices[1].message.content)

### O ChatModel é um componente LangChain então ele possui o protocolo invoke()

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, _ = initialize_llm()

logger.info("LLM and embeddings initialized successfully.")

In [ ]:
resposta = llm.invoke("Olá como você está e o que você é capaz de fazer?")
     
pretty_print(resposta)  

#### Criando a conversa. 

Lembrando que os ChatModels recebem como entrada uma lista de mensagem. Assim o LangChain automaticamente converte isso na estrutura que o modelo LLM precisa receber para responder.

In [ ]:

# Forma 1 de escrever:
mensagens = [
			 SystemMessage(content="Você é um especialista em astrofísica."),
			 HumanMessage(content="Qual a distancia do sol até a terra?"),
			 AIMessage(content="O Sol está a 49.600.000 km de distância da Terra."),
			 HumanMessage(content="E a distância da terra até marte?"),
]

# Forma 2 de escrever:
# mensagens = [
# 			 ("system", "Você é um especialista em astrofísica."),
# 			 ("user", "Qual a distancia do sol até a terra?"),
# 			 ("assistant", "O Sol está a&nbsp;49.600.000 km de distância da Terra."),
#            ("user", "E a distância da terra até marte?"),
# ]

# Como a entrada do usuário é a ultima mensagem da lista, você pode dá invoke usando a lista de pensagens contendo o histórico de conversação.
resposta = llm.invoke(mensagens)

pretty_print(resposta)
 

print("-------------------------------------------------------------------")

Vamos agora criar um chat, ou seja, vamos criar uma lista que vai crescendo dinamicamente com a entrada do usuário simulando uma conversa com ChatGPT.

Vamos agora simular o streaming de dados dos modelos (quando compatível) onde cada token é gerado em tempo de execução. Vamos repetir o código anterior e ao invés de invoke, vamos chamar o modelo usando a função assíncrona astream:

#### Rodando modelo localmente

Vamos agora fazer um teste rodando o ollama local utilizando um container docker


```bash

docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

docker exec -it ollama bash
ollama pull llama3.2

 
```


In [ ]:
! curl http://localhost:11434/api/generate -d '{"model": "llama3.2", "prompt":"Quem é Linux Torvald?"}'

In [ ]:
! curl http://localhost:11434/api/chat -d '{ "model": "llama3.2", "messages": [ { "role": "user", "content": "Por que o céu é azul?" }]}'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

OllamaLLM(  
     base_url="http://localhost:11434",  
     model = "llama3.2",  # nome do modelo que deseja tem em sua máquina
     temperature = 0.3,  
     num_predict = 1000, # numero máximo de tokens
 )

template = """
Você é um especialista em música brasileira e tem amplo conhecimento sobre letras de músicas.

Trecho da música: {question}

Resposta: Dê continuidade a música e o final conte seu histórico."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model = "llama3.2") 

chain = prompt | model

response = chain.invoke({"question": "Ouviram do Ipiranga às margens plácidas"})
print(response)

Utilizando a versão via chat model

In [ ]:
from langchain_ollama import ChatOllama

chat = ChatOllama(model="llama3.2")

response = chat.invoke("Conte uma piada")
print(response.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Faça perguntas sobre a arquitetura de software, você ajudará o usuário com perguntas da area, e responderá sempre com o minimo de informações."),
    HumanMessage(content="O que é SOLID?")
]
 
response_stream = chat.stream(messages)
for stream in response_stream:
    print(stream.content, end='', flush=True)

Agora vamos apagar o container para não ficar ocupando espaço


```bash
docker container stop ollama
docker container rm ollama
```